In [3]:
using LammpsAnalysis 


In [4]:
push!(LOAD_PATH, "/uni-mainz.de/homes/ywitzky/phdscripts/Julia/LammpsAnalysis/src/")
using ProteinSequences

In [5]:
using OffsetArrays, Plots,Measures, Printf, JLD2, Match, Revise, LoopVectorization, ProfileView, StatsBase, DataStructures, LaTeXStrings, LsqFit
import Setup

In [6]:
IDR_Proteins = ["RS41_IDR2", "RS41_PHOS6_IDR2"] #
Proteins = ["RS31a", "RS41", "RS41_PHOS6", "RS41_IDR2"]#, "RS41_PHOS6_IDR2"]
Temperatures =[280, 300, 320] #, 340, 360]
BasePath="/localscratch/HPS_DATA/HPS-Alpha/SLAB/" ### oldData
BasePath="/localscratch/HPS_DATA/HPS-Alpha/Bulk/"
NewPath = "/localscratch/HPS_DATA/HPS-Alpha/Bulk/"
Seitenlaenge =Dict("RS31a"=> 325.,"RS41"=> 325., "RS41_PHOS6"=> 325., "RS40"=> 325., "RS41_IDR2"=> 270.,"RS41_PHOS6_IDR2"=> 270., "RS41_RRM"=>270., "RS31a_IDR2"=> 170., "RS31"=>270)
RestartRule = Dict(300=>280, 320=>280, 340=>320, 360=>340)

RunNum = Dict([("RS31a", 280)=> 2,("RS31a", 300)=> 2,("RS31a", 320)=>1 ,("RS31a", 340)=>1 ,("RS31a", 360)=> 2, ("RS41", 280)=> 1,("RS41", 300)=> 1,("RS41", 320)=>1 ,("RS41", 340)=>1 ,("RS41", 360)=> 1, 	("RS41_PHOS6", 280)=> 1,("RS41_PHOS6", 300)=> 1,("RS41_PHOS6", 320)=>1 ,("RS41_PHOS6", 340)=>2 ,("RS41_PHOS6", 360)=> 1,])
""
FolderNames = Dict([("RS41", 280) => "SLAB_SameBox",("RS41_PHOS6", 280) => "SLAB_SameBox", ("RS41", 300) => "SLAB_SameBox",("RS41_PHOS6", 300) => "SLAB_SameBox", ("RS41", 320) => "SLAB_SameBox",("RS41_PHOS6", 320) => "SLAB_SameBox", ("RS41_IDR2", 280) => "SLAB_SameBox",("RS41_IDR2", 300) => "SLAB_SameBox",("RS41_IDR2", 320) => "SLAB_SameBox",("RS41_PHOS6_IDR2", 280) => "SLAB_SameBox",("RS41_PHOS6_IDR2", 300) => "SLAB_SameBox",("RS41_PHOS6_IDR2", 320) => "SLAB_SameBox"]);

In [20]:
BasePath="/localscratch/HPS_DATA/HPS-Alpha/SLAB/SLAB_Restart_Calvados2/" ### oldData

Protein= Proteins[2]
Temp=300
prot2= LammpsAnalysis.initData(BasePath*"$(Protein)/$(Temp)K/", "$(Protein)_slab.lmp";Reparse=true)  

energy file step: 4274 , number of restarts: 33 vs. 4307 , 1//1


LammpsAnalysis.SimData{Float32, Int32}("RS41_slab", 1, 4339, 71400, 71200, 71000, 70800, 200, 22, 1, 1, 238, 300.0f0, 2, Float32[-162.5 162.5; -750.0 750.0; -162.5 162.5], Float32[325.0, 1500.0, 325.0], ["M',KPVFCGNFEYDARESDLERLFRKYGKVERVDMKAGFAFVYMEDERDAEDAIRALDRFEYGRTGRRLRVEWTKNDRGGAGRSGGSRRSSSGLRPSKTLFVINFDAQNTRTRDLERHFEPYGKIVNVRIRRNFAFIQYEAQEDATRALDATNSSKLMDKVISVEYAVKDDDSRGNGYSPERRRDRSPDRRRRSPSPYRRERGSPDYGRGASPVAHKRERTSPDYGRGRRSPSPYKRARLSPDYKRDDRRRERVASPENGAVRNRSPRKGRGESRSPPPYEKRRESRSPPPYEKRRESRSPPPYEKRRERSRSRSKSSPENGQVESPGQIMEVEAGRGYDGADSPIRESSPSRSPPAEE',", "MKPVFCGNFEYDARESDLERLFRKYGKVERVDMKAGFAFVYMEDERDAEDAIRALDRFEYGRTGRRLRVEWTKNDRGGAGRSGGSRRSSSGLRPSKTLFVINFDAQNTRTRDLERHFEPYGKIVNVRIRRNFAFIQYEAQEDATRALDATNSSKLMDKVISVEYAVKDDDSRGNGYSPERRRDRSPDRRRRSPSPYRRERGSPDYGRGASPVAHKRERTSPDYGRGRRSPSPYKRARLSPDYKRDDRRRERVASPENGAVRNRSPRKGRGESRSPPPYEKRRESRSPPPYEKRRESRSPPPYEKRRERSRSRSKSSPENGQVESPGQIMEVEAGRGYDGADSPIRESSPSRSPPAEE", "MKPVFCGNFEYDARESDLERLFRKYGKVERVDMKAGFAFVYMEDERDAEDAIRALDRFEYGRTGRRLRV

In [8]:
using Mmap

In [19]:
function parseXYZ!(Sim::LammpsAnalysis.SimData)
    traj=open(Sim.TrajectoryFile, "r")
    
    xnew  = zeros(eltype(Sim.x), Sim.NAtoms)
    ynew  = zeros(eltype(Sim.x), Sim.NAtoms)
    znew  = zeros(eltype(Sim.x), Sim.NAtoms)

    step=0
    atom_cnt=0
    write_step=0
    for line in eachline(traj)    
        nspaces=count(" ", line)
        if (step%Sim.StepFrequency==0)
            if atom_cnt ==0 println("Step: $write_step") end
            if atom_cnt==0 ### Atoms. Timestep: xxxxx
                if write_step!=0
                    Sim.x[:, write_step] .= xnew
                    Sim.y[:, write_step] .= ynew
                    Sim.z[:, write_step] .= znew
                end
                write_step+=1
                atom_cnt+=1
                continue
            elseif nspaces==3 
                
                lines = Base.split(line)
                if lines[1]=="Atoms."
                    continue
                end                
                #=
                split = parse.(eltype(Sim.x),lines)

                xnew[atom_cnt] = split[2]
                ynew[atom_cnt] = split[3]
                znew[atom_cnt] = split[4]
                =#
    
                atom_cnt+=1
            end
        end
        if nspaces==0
            atom_cnt=0
            step+=1
        end
        
    end
    close(traj)

    return write_step-1
end

parseXYZ! (generic function with 1 method)

In [10]:
function readXYZ!(Sim::LammpsAnalysis.SimData, TrajectoryFile::String, EnergyFile::String, Minimize=false, NumSteps=-1, Delimiter=" ")
    LammpsAnalysis.readEnergyFile(Sim; EnergyFile=EnergyFile, NumSteps=NumSteps)

    ### Sim.StepFrequency is for data that will be reduceed, Sim.reduce sets discrepancy between energy data and .xyz created by presorting
    if Sim.TrajectoryFile[end-3:end] =="xyz"
        traj=open(TrajectoryFile, "r")
        Sim.NAtoms= parse(Int32,readline(traj))
    elseif Sim.TrajectoryFile[end-3:end] =="xtc"
        (_, xtc) = xtc_init(TrajectoryFile)
        Sim.NAtoms = xtc.natoms
        close(xtc)
    end

    Sim.xio= open(Sim.xFilePath,"w+")
    Sim.yio= open(Sim.yFilePath,"w+")
    Sim.zio= open(Sim.zFilePath,"w+")

    Sim.x =  Mmap.mmap(Sim.xio, Matrix{eltype(Sim.x)}, (Sim.NAtoms,Sim.NSteps))
    Sim.y =  Mmap.mmap(Sim.yio, Matrix{eltype(Sim.x)}, (Sim.NAtoms,Sim.NSteps))
    Sim.z =  Mmap.mmap(Sim.zio, Matrix{eltype(Sim.x)}, (Sim.NAtoms,Sim.NSteps))

    ### actually read the data
    if Sim.TrajectoryFile[end-3:end] ==".xyz"
        if Delimiter==","
            Sim.NSteps= parseXYZ_CSV!(Sim) 
        else
            Sim.NSteps= parseXYZ!(Sim) 
        end
    elseif Sim.TrajectoryFile[end-3:end] ==".xtc"
    LammpsAnalysis.parseXTC!(Sim)
    else
        println("Can not read file format: $( Sim.TrajectoryFile[end-3:end])")
    end

    ### sync RAM to disk before closing
    Mmap.sync!(Sim.x)
    Mmap.sync!(Sim.y)
    Mmap.sync!(Sim.z)
    close(Sim.xio)
    close(Sim.yio)
    close(Sim.zio)
end

readXYZ! (generic function with 4 methods)

In [18]:
LammpsAnalysis.readEnergyFile(prot2;EnergyFile=prot2.EnergyFile)

energy file step: 4274 , number of restarts: 33 vs. 4307 , 1//1


4317

In [ ]:
readXYZ!(prot2, prot2.TrajectoryFile, prot2.EnergyFile)

energy file step: 4274 , number of restarts: 33
Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Step: 50
Step: 51
Step: 52
Step: 53
Step: 54
Step: 55
Step: 56
Step: 57
Step: 58
Step: 59
Step: 60
Step: 61
Step: 62
Step: 63
Step: 64
Step: 65
Step: 66
Step: 67
Step: 68
Step: 69
Step: 70
Step: 71
Step: 72
Step: 73
Step: 74
Step: 75
Step: 76
Step: 77
Step: 78
Step: 79
Step: 80
Step: 81
Step: 82
Step: 83
Step: 84
Step: 85
Step: 86
Step: 87
Step: 88
Step: 89
Step: 90
Step: 91
Step: 92
Step: 93
Step: 94
Step: 95
Step: 96
Step: 97
Step: 98
Step: 99
Step: 100
Step: 101
Step: 102
Step: 103
Step: 104
Step: 105
St

LoadError: BoundsError: attempt to access 71400×4317 Matrix{Float32} at index [1:71400, 4318]